# Databricksにおける機械学習モデルの構築と管理

Databricks の `scikit-learn` ライブラリを使用して機械学習分類モデルを構築する方法について説明します。

目標は、ワインが「高品質」と見なされるかどうかを予測する分類モデルを作成することです。 データセットは、さまざまなワインの 11 の特徴 (アルコール含有量、酸度、残留糖など) と、1 から 10 の品質ランキングで構成されています。

この例では、[MLflow](https://docs.databricks.com/aws/ja/mlflow/) を使用してモデル開発プロセスを追跡します。

[チュートリアル: Databricks で初めての機械学習モデルを構築する \| Databricks Documentation](https://docs.databricks.com/aws/ja/getting-started/ml-get-started)

## ライブラリのインストール

In [0]:
%pip install -U mlflow
%restart_python

## モデルレジストリ、カタログ、スキーマの設定

モデルレジストリとしてUnity Catalogを使用するように、MLflowクライアントを構成する必要があります。

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")

また、モデルが登録されるカタログとスキーマも設定する必要があります。

In [0]:
# 必要に応じて、「workspace」と「default」を、必要な権限を持つカタログおよびスキーマに置き換えてください。
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"

## EDA(探索的データ分析)

機械学習モデルを構築する際には、トレーニングに使用するデータの特徴や傾向を把握することが重要です。Databricksの機能やライブラリを活用して、いくつかの切り口でデータを可視化して、データへの理解を深めます。これが探索的データ分析(EDA: Exploratory Data Analysis)です。

In [0]:
import pandas as pd

# CSVファイルからデータを読み込み
white_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-white.csv", sep=";")
red_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-red.csv", sep=";")

# フラグを追加
red_wine['is_red'] = 1
white_wine['is_red'] = 0

# データフレームを結合
wine_data = pd.concat([red_wine, white_wine], axis=0)

データの中身を確認します。Databricksでは`display`関数を用いることで、簡単にデータを可視化することができます。

In [0]:
display(wine_data)

目的変数の`quality`のヒストグラムをプロットします。

In [0]:
import seaborn as sns
wine_data = wine_data.sort_values("quality")
sns.histplot(data=wine_data.quality)

## データを読み込み、Unity Catalogテーブルを作成

この例では、`databricks-datasets`で使用できる2つのCSVファイルを使用します。

以下のコードは、次のことを行います。

1. `winequality-white.csv`と`winequality-red.csv`からデータを読み取り、Sparkデータフレームに読み込みます。先ほどはPandasデータフレームを使いましたが、Pandasデータフレームは直接テーブルに書き込めないため、ここではSparkデータフレームを使います。
1. 列名のスペースをアンダースコアに置き換えて、データをクリーンアップします。
1. データフレーム を Unity Catalogの `white_wine` テーブルと `red_wine` テーブルに書き込みます。データを Unity Catalog に保存すると、データが保持され、他のユーザーと共有する方法を制御できます。

In [0]:
white_wine = spark.read.csv("/databricks-datasets/wine-quality/winequality-white.csv", sep=';', header=True)
red_wine = spark.read.csv("/databricks-datasets/wine-quality/winequality-red.csv", sep=';', header=True)

# 列名からスペースを削除
for c in white_wine.columns:
    white_wine = white_wine.withColumnRenamed(c, c.replace(" ", "_"))
for c in red_wine.columns:
    red_wine = red_wine.withColumnRenamed(c, c.replace(" ", "_"))

# テーブル名を定義
red_wine_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.red_wine"
white_wine_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.white_wine"

# Unity Catalogのテーブルに書き込み
spark.sql(f"DROP TABLE IF EXISTS {red_wine_table}")
spark.sql(f"DROP TABLE IF EXISTS {white_wine_table}")
white_wine.write.saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.white_wine")
red_wine.write.saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.red_wine")

テーブルを確認しましょう。以下のコマンドを実行して表示されるリンクをクリックしてください。

In [0]:
displayHTML(f"<a href='/explore/data/{CATALOG_NAME}/{SCHEMA_NAME}'>テーブルを確認</a>")

## データの前処理と分割

このステップでは、前のステップで作成した Unity Catalog テーブルから Pandas データフレーム にデータを読み込み、データを前処理します。 このセクションのコードでは、次のことを行います。

1. データを Pandas データフレームとしてロードします。
1. 赤ワインと白ワインを区別するために各 にBoolean 列を追加し、2つのデータフレームを新しいデータフレーム`データフレーム データフレーム`data_df`に結合します。
1. データセットには、ワインを 1 から 10 まで評価する `quality` 列が含まれており、10 は最高品質を示します。 このコードは、この列を 2 つの分類値に変換します。"True" は高品質のワイン (`quality >= 7`) を示し、"False" は高品質ではないワイン (`quality < 7`) を示します。
1. データフレーム を個別のトレーニング データセットとテスト データセットに分割します。

In [0]:
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.ensemble

import matplotlib.pyplot as plt

In [0]:
# Unity CatalogからデータをPandasデータフレームとして読み込む
white_wine = spark.read.table(f"{CATALOG_NAME}.{SCHEMA_NAME}.white_wine").toPandas()
red_wine = spark.read.table(f"{CATALOG_NAME}.{SCHEMA_NAME}.red_wine").toPandas()

# 赤ワイン・白ワインのブール型フィールドを追加
white_wine['is_red'] = 0.0
red_wine['is_red'] = 1.0
data_df = pd.concat([white_wine, red_wine], axis=0)

# ワインの品質に基づいて分類ラベルを定義
data_labels = data_df['quality'].astype('int') >= 7
data_df = data_df.drop(['quality'], axis=1)

# 80/20で学習用・テスト用に分割
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
  data_df,
  data_labels,
  test_size=0.2,
  random_state=1
)

## 分類モデルのトレーニング

この手順では、デフォルト アルゴリズム設定を使用して勾配ブースティング分類器をトレーニングします。 次に、結果のモデルをテストデータセットに適用し、 [ROC(Receiver Operating Characteristic)曲線](https://ja.wikipedia.org/wiki/%E5%8F%97%E4%BF%A1%E8%80%85%E6%93%8D%E4%BD%9C%E7%89%B9%E6%80%A7)のAUC(Area Under Curve)を計算、ログ、および表示して、モデルのパフォーマンスを評価します。

In [0]:
# MLflow の自動ログ記録を有効にします
mlflow.autolog()

モデルトレーニングを実行します。

In [0]:
white_dataset = mlflow.data.load_delta(table_name=f"{CATALOG_NAME}.{SCHEMA_NAME}.white_wine", version="0")
red_dataset = mlflow.data.load_delta(table_name=f"{CATALOG_NAME}.{SCHEMA_NAME}.red_wine", version="0")

In [0]:
with mlflow.start_run(run_name='gradient_boost') as run:
    model = sklearn.ensemble.GradientBoostingClassifier(random_state=0)

    # モデル、パラメータ、学習メトリクスは自動でトラッキングされます
    model.fit(X_train, y_train)

    predicted_probs = model.predict_proba(X_test)
    roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
    roc_curve = sklearn.metrics.RocCurveDisplay.from_estimator(model, X_test, y_test)

    # ROC曲線のプロットをファイルに保存
    roc_curve.figure_.savefig("roc_curve.png")

    # テストデータのAUCスコアは自動で記録されないため、手動で記録
    mlflow.log_metric("test_auc", roc_auc)

    # ROC曲線画像ファイルをアーティファクトとして記録
    mlflow.log_artifact("roc_curve.png")

    # トレーニングデータも記録します
    mlflow.log_input(white_dataset, context="training")
    mlflow.log_input(red_dataset, context="training")

    print("テストAUC: {}".format(roc_auc))

## MLflowでのエクスペリメントランの参照

MLflowのエクスペリメントトラッキングは、モデルを反復的に開発するときにコードと結果をログに記録することで、モデル開発を追跡するのに役立ちます。

上のセルの出力に表示されている**View Logged Model**のリンクをクリックします。

## モデルをUnity Catalogに登録

ここまでは試行錯誤を通じた**実験**段階です。モデルが目標精度、性能を達成した場合には本番運用に進むことになります。DatabricksのUnity Catalogは本番運用におけるモデル管理の機能である**モデルレジストリ**を提供しています。

また、上で記録したモデルにはGUIだけではなく、Pythonからもアクセスできるのでモデルの運用を自動化することも可能です。

In [0]:
# テストAUCでランをソートします。同点の場合は最新のランを使用します。
best_run = mlflow.search_runs(
  order_by=['metrics.test_auc DESC', 'start_time DESC'],
  max_results=10,
).iloc[0]
print('ベストラン')
print('AUC: {}'.format(best_run["metrics.test_auc"]))
print('推定器数: {}'.format(best_run["params.n_estimators"]))
print('最大深さ: {}'.format(best_run["params.max_depth"]))
print('学習率: {}'.format(best_run["params.learning_rate"]))

Pythonを用いて上の分類モデルをUnity Catalogに記録します。

In [0]:
model_uri = 'runs:/{run_id}/model'.format(
    run_id=best_run.run_id
  )

mlflow.register_model(model_uri, f"{CATALOG_NAME}.{SCHEMA_NAME}.wine_quality_model")

上のリンクをクリックして、**依存関係**のタブをクリックしてみましょう。

このあとはテストを経て本番環境へのデプロイに進むことになります。興味のある方は[Mosaic AI Model Serving を使用したモデルのデプロイ](https://docs.databricks.com/aws/ja/machine-learning/model-serving/)をご覧ください。